<img style="float: right;" src="files/UNESCAP75.png" width=300 height=300 />

# ESCAP Carbon Pricing simulation tool

In [1]:
# Import modelflow
#%load_ext autoreload
#%autoreload 2

In [2]:
%matplotlib inline

from ipywidgets import interact,Dropdown,Checkbox,Select
from IPython.display import display, clear_output,Latex, Markdown
import pandas as pd

import modeljupyter as mj
from modelvis import waterplot

from modelinvert import targets_instruments
from modelclass import model
model.widescreen()
model.scroll_off()
model.modelflow_auto()

# First the model is loaded

In [3]:
# Load model and data, and run 

if not 'masia' in locals():
    print('The model is beeing loaded into the notebook.')
    print('It may take some time, depending on the server load.')
    masia,baseline = model.modelload('Asia_19nov.pcim',run=1,silent=1)
    print('The model is now loaded')

from inject import inject
inject(masia)    

The model is beeing loaded into the notebook.
It may take some time, depending on the server load.
The model is now loaded


# Hints

If loading is not happening, or if you have other problems restart the Notebook. 

To restart press **cell**>**run all** in the menu 

# You can select a country for which to make simulations
Then input a scenario for the country, run the scenario and show the results  

# Business as usual 
Business as usual emissions are based on an assumption of NO CHANGE in the energy mix after 2020


In [4]:
lookat_countries = sorted(masia.countries_ASEAN+
('FSM RUS BGD IRN TUR ARM MDV KAZ TLS IND UZB BTN WSM GEO AZE'.split()))
tup_countries = [(masia.iso_dict[c],c) for c in lookat_countries]
selected = Select(options = tup_countries,description='Select country',style={'width':'60%','description_width':'40%'})
display(selected)

selected_country = selected.value.split()

def show(g):
    clear_output()
    display(selected)
    # print(g)
    selected_country = g['new'].split()
    
    masia.keep_solutions = {}
    masia.keep_solutions["Business as usual"]= baseline.copy()

    # Here we define the input widgets
    from modelwidget import sumslidewidget,slidewidget, updatewidget, tabwidget,sheetwidget
    outvar = 'GDI GCARBR CO2 LNN'.split()

    #  masia.countries_GCARBR_A is list of all countries in the model with a countries_GCARBR_A 
    #  masia.countries_ASEAN is list of all countries in ASEAN 

    slidedef_ambition_countries = {  "Climate policy ambition": 
                          {'var': f'{country}_AMB', 'value': masia.basedf.loc[2021,f'{country}_AMB'] ,'min':1,'max':5,'op':'=','step':0.1,'dec':1} 
                          for country in  selected_country}
    slidedef_enabling_countries = {'Enabling factors for climate action' : 
                          {'var': f'{country}_ENAB', 'value': masia.basedf.loc[2021,f'{country}_ENAB'],'min':1,'max':5,'op':'=','step':0.1,'dec':1} 
                          for country in  selected_country}

    updatevar = [f'{country}_GCARBR_A' for country in selected_country]
    smalldf = baseline.loc[2022:2040, updatevar]
    c = selected_country[0]
    displayvar = f'{c}_CO2 {c}_GCARBR {c}_INFL {c}_GDNRATIO {c}_YER {c}_PM25 {c}_INFL'\
    f'{c}_URX {c}_HEAD55 {c}_GINI_DISP {c}_TFP'

    tabsharedef = {masia.iso_dict.get(country,country) : sumslidewidget(
                    {'Environmental protection':{'var': f'{country}_SHAREE', 'value': 0.25,'min':0.0,'max':1,'op':'=','step':0.05,'dec':2} ,
                     'Health'                  :{'var': f'{country}_SHAREH', 'value': 0.25,'min':0.0,'max':1,'op':'=','step':0.05,'dec':2} ,
                     'Social protection'       :{'var': f'{country}_SHARESP','value': 0.25,'min':0.0,'max':1,'op':'=','step':0.05,'dec':2} ,
                     'Education'               :{'var': f'{country}_SHAREED','value': 0.0,'min':0.0,'max':1,'op':'=','step':0.05,'dec':2} ,
                     'Connectivity'            :{'var': f'{country}_SHARECX','value': 0.0,'min':0.0,'max':1,'op':'=','step':0.05,'dec':2} ,
                     'Energy efficiency gains' :{'var': f'{country}_SHAREX', 'value': 0.25,'min':0.0,'max':1,'op':'=','step':0.05,'dec':2} 
                    } , expname='Share (the sum can max be 1.0) of Carbon revenue spent on:', maxsum=1.0)  

                   for country in selected_country }


    tabdef = {
              'Individual tax rates'      : (sheet1 :=  sheetwidget(df_var = smalldf,transpose=0,trans=masia.get_a_country)),
              'Use of carbon tax revenue' : tabwidget(tabdefdict = tabsharedef,tab=True,selected_index = 0),
              'Ambition and enabling'    : slidewidget({**slidedef_ambition_countries,**slidedef_enabling_countries},expname='On a scale from 1 to 5')
                }

    select_dict = {c:d for c,d in masia.iso_dict.items() if c  in selected_country }
    prefix_dict = select_dict

    wtab = tabwidget(tabdefdict = tabdef,tab=False)
    xx = updatewidget(masia,wtab,lwreset=True,varpat = displayvar,outputwidget='keep_viz_prefix',
                      prefix_dict={},display_first = selected,vline=[(2021,'')],relativ_start=-10) # output widget = either keep_viz or jupviz 
    
    xx.show(None)
    #display(selected)
    #display(xx.wtotal)
    


selected.observe(show,names='value',type='change')
    

Select(description='Select country', options=(('Armenia', 'ARM'), ('Azerbaijan', 'AZE'), ('Bangladesh', 'BGD')…